# Exemple d'algorithmes génétiques avec un master mind 
## Objectif :
présenter simplement les algorithmes génétiques à l'aide d'un exemple simple, un master mind !
Ce fichier sera le premier d'une songue série de notebooks sur les algorithmes génétiques. 
Il pourra m'aider à l'analyse des différentes techniques et à leurs cas d'utilisation.
L'objectif est pour la machine de deviner une combinaison en un minimum d'iterations. 
Moins il y a d'iterations, plus l'algorithme est performant.Dans le mastermind original, l'utilisateur a perdu s'il ne trouve pas la combinaison gagnante au bout d'un certain nombre d'iterations. 
Dans l'exemple ici, la machine ne peut pas perdre. L'objectif était de tester la robustesse des algorithmes.
## Règle du mastermind
Le mastermind se joue à 2 joueurs, sur un plateau de 4 colonnes et de 12 rangés et se joue avec des billes de 4 couleurs différentes.Le mastermind possède une combinaison secrète de 4 billes de couleurs, et l'objectif du joueur est de deviner avant la 13e tentative la combinaison secrète. 
Pour l'aider, il lui est révélé né nombre de billes posées au bon endroit et de la bonne couleur, sans lui reveler lesquels sont correctes.
## Ce qui est présenté dans ce notebook
- La notion de fitness
- La notion de population
- La notion d'individus
- La notion de chromosomes
- La notion de genes 
## Concepts avancée dans ce notebook
- Les types de sélections    
    - Sélection par roulette    
    - Sélection par rang    
    - Sélection par tournoi    
    - Sélection aléatoire
- La notion d'élitisme
- Les types de reproduction d'individus    
    - One point cross-over    
    - Two-point cross-over    
    - Uniforme Cross-Over

In [17]:
# Importation des librairies utilisés
import random
defaultProbaPerLetter = 10
batch_size = 30

In [18]:
# Definition du plateau
class Plateau:
    def __init__(self, secret):
        self.secret = secret
        self.secretLen = len(secret);
        #initialisation des genes (probabilité de 10 chaqu'un)
        self.caracteres = "azertyuiopqsdfghjklmwxcvbn0123456789_'";
        self.genesDict = []
        self.totalGenesCount = defaultProbaPerLetter*len(self.caracteres)
        # remplissage du tableau de prédiction avec la valeur par defaut.
        for i in range (0,self.secretLen):
            self.genesDict.append({});
            for l in self.caracteres:
                self.genesDict[i][l] = defaultProbaPerLetter;
        
    def printDict(self):
        print(self.genesDict)
        
    '''
    Cette fonction sert à mémoriser les résultats générés précedement. il enregistre pour chaque 
    case les résultats obtenus lors des precedentes tentatives. Cela permet à l'algorithme d'affiner lentement 
    sa prédiction pour tendre vers la bonne solution.
    '''
    def evolveGenes(self, individu, fitness):
        if (fitness == 0):
            return;
        index = 0
        for l in individu:
            self.genesDict[index][l]+= fitness
            index+=1;
        self.totalGenesCount += fitness

Il existe aussi une methode appelée uniform crossover, qui change aléatoirement les genes, dans "paquets ou coupures"distinctes. Un exemple avec les deux parents ----- et +++++ peut donner -+-+-. 
__Cet exemple n'est pas encore implémenté dans l'algorithme.__
<hr>
Voici la definition du joueur.

In [19]:
# Definition du joueur. 
# Il va jouer un coup, et le changer (muter) pour chercher une meilleur combinaision
class Joueur:
    def __init__(self, plateau):
        self.p = plateau;
        self.crossoverOne = True;
        
    # Generation d'un individu aléatoire.
    def generateRandomPlay(self):
        individu = "";
        for i in range(0,self.p.secretLen):
            lettre = p.caracteres[random.randint(0, len(p.caracteres)-1)];
            individu += lettre
        return individu;
    
    # generation d'un individu dont les genes sont probabilisé par les résultats précedents
    def generatePlay(self):
        individu = "";
        for i in range(0,self.p.secretLen):
            letterIndex = random.randint(0, p.totalGenesCount)
            for l in p.genesDict[i]:
                if (letterIndex - int(p.genesDict[i][l]) <= 0):
                    individu+=l
                    break
                else:
                    letterIndex -= p.genesDict[i][l]
        return individu;
    
    # generation d'une population de 10 individus (par defaut) avec des genes aléatoires.
    def generatePopulation(self):
        pop = [];
        for i in range(0,batch_size):
            play = self.generateRandomPlay();
            pop.append(play);
        return pop;
    
    '''
    Cette fonction genere des enfants qui possede une partie des genes de son parent.
    la generation des genes des enfant est généré par un one point crossover, ou un two point crossover.
    Le principe de mutation n'est pas introduit ici. il a pour principe de changer au hasard un gene sans 
    tenir compte de sa probabilité. Les mutations doivent etre rarent, et permettent de limiter la convergence
    de l'algorithme.
    '''
    def generatePopulationFromParent(self, parent):
        pop = [];
        for i in range(0,batch_size):
            play = self.generatePlay();
            if (self.crossoverOne):
                play = self.onePointCrossover(play);
            else:
                play = self.twoPointCrossover(play);
            pop.append(play);
        return pop;
        
    '''
    La fitness est la valeur utilisée par l'algorithme pour estimer ou noter le resultat d'un individu
    sur l'objectif à atteindre. Dans notre cas, chaque lettre correctement placée compte comme 1 point.
    La fitness maximale est donc la taille du mot secret !
    '''
    def getFitness(self, individu):
        fitness = 0;
        for i in range(0,self.p.secretLen):
            if(self.p.secret[i] == individu[i]):
                fitness += 1;
        return fitness;
    
    # retourne l'index dans la population de l'element avec la meilleur fitness 
    # (la plus haute correspondence au mot secret)
    def getFittestIndex(self, population):
        index = 0
        fittest = self.getFitness(population[index])
        for i in range(0,self.p.secretLen):
            fit = self.getFitness(population[i])
            if (fittest < fit):
                index = i;
                fittest = fit;
        return index;
    
    ''' 
    Le one point crossover est une methode de generation d'enfant.
    Admettons un parent avec le chromosome (ensemble de genes) suivant : '-----''
    Et admettons un individu généré avec les genes suivant : '+++++'
    Le one point crossover va generer l'enfant des deux parents en prennant 
    un index aléatoirement, puis en melangeant les genes du premier parent avec le second
    sur cet index.
    exemple avec l'index 2, l'enfant des deux parents ci dessus donne : '--+++'.
    '''
    def onePointCrossover(self, individu):
        side = random.randint(0, 1)
        i = random.randint(1,self.p.secretLen-1)
        play = self.generateRandomPlay()
        if (side == 0):
            ret = individu[:i] + play[:self.p.secretLen-i]
        else:
            ret = play[:i] + individu[i-self.p.secretLen:] 
        return ret;
        
    '''
    Le two point crossover sert egalement à melanger les genes de deux parents, mais cette fois-çi avec deux
    index.
    Donc pour les mêmes parents ----- et +++++ avec les index 2 et 4, l'enfant donnera : --++-.
    '''
    def twoPointCrossover(self, individu):
        i = random.randint(0,self.p.secretLen-1)
        size = random.randint(1,self.p.secretLen-i)
        play = self.generateRandomPlay()
        ret = individu[0:i] + play[i:i+size] + individu[i+size:self.p.secretLen]
        return ret;

In [20]:
# Initialisation du jeu
# Definir 5 combinaisons de 38 possibilités correspond à une chance sur 3.64e+26 de tomber sur la bonne.
p = Plateau("wha0u");
j = Joueur(p);


Ici, nous allons selectionner le meilleur parent, et le faire se reproduire pour former la prochaine generation. C'est à mon sens la meilleure solution dans ce cas, car il n'existe qu'une seule solution, qui est accessible via un seul chemin. Résuire le nombre de parents permet donc de converger plus rapidement vers la meilleure solution.

Voici l'algorithme principal de la résolution du mastermind. un bref résumé de l'algorithme donne :
- Création d'une population aléatoire
- Calcul de la fitness de la population, et récuperation du meilleur individu
- Jusqu'a ce que la solution soit trouvé, faire :
    - Selectionner un parent depuis la population, 
    - Appliquer un crossover et generer une nouvelle population
    - Calculer la fitness de la nouvelle population

In [ ]:
# algorithme principal :
#création d'une population de 10 individus pour le demarrage :
pop = j.generatePopulation();
fittestIndex = j.getFittestIndex(pop)
fitness = j.getFitness(pop[fittestIndex])
print(pop[fittestIndex] + "   --fitness : "+str(fitness));
p.evolveGenes(pop[fittestIndex], fitness)
p.printDict()
i=1;
while (fitness < p.secretLen):
    pop = j.generatePopulationFromParent(pop[fittestIndex])
    fitness = j.getFitness(pop[fittestIndex])
    p.evolveGenes(pop[fittestIndex], fitness)
    print(pop[fittestIndex] + "   --fitness : "+str(fitness));
    i+=1;

print("!! Trouvé !! - "+ pop[fittestIndex]+"  //  tentatives : "+str(i))

Dans l'algorithme principal au dessus, La selection du parent se fait simplement par le fair que le meilleur l'emporte. Cependant il existe plusieurs autres types de selection de parents. Nous allons en voir quelques exemples en dessous.
<hr>

# La selection par roulette
Le principe est simple. Les individus avec le plus de fitness ont plus de chance d'etre gardé pour la prochaine production de la population. illustration simple en image :

<img src="https://www.researchgate.net/profile/Manfred_Breit/publication/237507026/figure/fig7/AS:298797780488200@1448250350801/Roulette-wheel-selection-using-fitness-weighted-probability.png" alt="roulette wheel selection;">

[Source de l'image](https://www.researchgate.net/figure/Roulette-wheel-selection-using-fitness-weighted-probability_fig7_237507026)

Voici un algorithme pour le mettre en place :

In [21]:
#TODO

<hr>

# Sélection par rang
<img src="https://www.tutorialspoint.com/genetic_algorithms/images/rank_selection.jpg" alt="rank selection;">
Avec cette selection. on garde un nombre fixe d'individus issu de la population ayant la meilleur fitness. une fois ces individus en nôtre pocession, on en selectionne un au hasard, puis on regénere la population.

Cette methode fonctionne avec des elements posedant une fitness negative, mais à l'inconvenient de perdre en selection naturelle rapidement, et ainsi donc privilegier les mauvais parents lors de la génération.

exemple d'algorithme ci dessous :

In [22]:
#TODO

In [ ]:
individu = j.generateRandomPlay();
fitness = j.getFitness(individu);
pop = j.generatePopulation();
index = j.getFittestIndex(pop);
#print(individu + "   -- fitness : "+str(fitness));
print(pop)
print (index)
print(j.getFitness(pop[index]))